# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [ ]:
## NOTEBOOK CHANGES OVERVIEW
# Added config for hostname of desktop
# Changed run_robot function to work with changes
# Results in successful, but mediocre performance of the controller

# verifying pathlib is clean
import pathlib
print(pathlib.__file__)

/usr/lib/python3.12/pathlib.py


In [2]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import math
import socket
import pathlib
import time
import torch
import numpy as np

from robot.al5d_position_controller import PositionController, RobotPosition

hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    preferred_camera = "dev0"
    try:
        from robotcontrol.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxend module not found, cannot use gamepad")
        available_gamepad = False
    available_robot = True
elif hostname == "szenes.local":
    available_gamepad = False
    available_robot = False
    preferred_camera = "dev0"
elif hostname == "glassy":
    available_gamepad = False
    available_robot = False
    preferred_camera = "dev0"
# I changed the hostname of the desktop to labDesktop0
elif hostname == "labDesktop0":
    available_gamepad = True
    preferred_camera = "dev0"
    try:
        from robotcontrol.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxend module not found, cannot use gamepad")
        available_gamepad = False
    available_robot = True
else:
    available_gamepad = False
    available_robot = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot


from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

Hostname is labDesktop0
/usr/lib/python3.12/pathlib.py
***ExpRun**: Loading pointer config file:
	/home/al5d/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	~/WORK/BerryPicker/cfg/settings.yaml
Using device: cuda


In [3]:
# CHANGES
# Added this cell to follow continuity of loading in paths from previous notepads

# *** Initialize the variables with default values 
# *** This cell should be tagged as parameters     
# *** If papermill is used, some of the values will be overwritten 

# If it is set to true, the exprun will be recreated from scratch
creation_style = "exist-ok"

# If not None, set an external experiment path
external_path = None
# If not None, set an output path
data_path = None

experiment = "behavior_cloning"
# run = "bc_mlp_00"
# run = "bc_lstm_00"
# run = "bc_lstm_resid_00"
run = "bc_lstm_resid_00_validation_leaf"
# run = "bc_lstm_mdn_00"
# exp = Config().get_experiment(experiment, run)

In [4]:
# CHANGES
# Added this cell to follow continuity of loading in configs from previous notepads

if external_path:
    external_path = pathlib.Path(external_path)
    assert external_path.exists()
    Config().set_experiment_path(external_path)
    Config().copy_experiment("sensorprocessing_conv_vae")
    Config().copy_experiment("robot_al5d")
    Config().copy_experiment("demonstration")
    Config().copy_experiment("behavior_cloning")
if data_path:
    data_path = pathlib.Path(data_path)
    assert data_path.exists()
    Config().set_experiment_data(data_path)

exp = Config().get_experiment(experiment, run, creation_style=creation_style)
pprint.pprint(exp)
exp_sp = Config().get_experiment(exp["exp_sp"], exp["run_sp"])
exp_robot = Config().get_experiment(exp["exp_robot"], exp["run_robot"])

***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_resid_00_validation_leaf successfully loaded
Experiment:
    batch_size: 1
    control_size: 6
    controller: bc_LSTM_Residual
    controller_file: controller.pth
    data_dir: /home/al5d/WORK/BerryPicker/data/behavior_cloning/bc_lstm_resid_00_validation_leaf
    epochs: 300
    exp_bc: behavior_cloning
    exp_robot: robot_al5d
    exp_robot_controller: robot_al5d
    exp_run_sys_indep_file: /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/behavior_cloning/bc_lstm_resid_00_validation_leaf.yaml
    exp_sp: sensorprocessing_conv_vae
    experiment_name: behavior_cloning
    hidden_size: 64
    loss: MSELoss
    name: bc_lstm_resid_00_validation_leaf
    optimizer: Adam
    optimizer_lr: 0.001
    run_bc: bc_lstm_resid_00_validation_leaf
    run_name: bc_lstm_resid_00_validation_leaf
    run_robot: position_controller_00
    run_robot_controller: position_controller_00
    run_sp: sp_vae_128_300epochs_val

In [5]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
# run = "runbc_bc_lstm_0001"
exp = Config().get_experiment(experiment, run)

# note that we are loading the exprun for the robot controller even if the robot is not connected
exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])

if hostname == "glassy":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_glassy")
elif hostname == "raven":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_raven")
elif hostname == "szenes.local":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_szenes")
elif hostname == "labDesktop0":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_labDesktop0")
else:
    exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

['bc_mlp_00',
 'mdn_for_bc_00',
 'bc_verify_mlp_00',
 'bc_lstm_00',
 'bc_lstm_resid_00_validation',
 'bc_verify_lstm_00',
 'bc_run_00',
 'mdn_00',
 'bc_compare_00',
 'bc_lstm_mdn_00',
 'bc_verify_lstm_resid_00',
 'bc_verify_lstm_mdn_00',
 '_defaults_behavior_cloning',
 'bc_lstm_resid_00',
 'bc_lstm_resid_00_validation_leaf']
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_resid_00_validation_leaf successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_labDesktop0 successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_resid_00_validation_leaf successfully loaded
***ExpRun**: Configuration for exp/run: sensorprocessing_conv_vae/sp_vae_128_300epochs_validation

In [6]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))   

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
Try out the backup /dev/ttyUSB0
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
cap0 works


<All keys matched successfully>

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

In [ ]:
def safe_robot_control(a, robot_controller, exp_robot_controller):
    """Moves the robot to the position, with various safety controler"""
    # Controller output order will be {'height', 'distance', heading', 'wrist_angle', 'wrist_rotation', 'gripper'}
    # As shown in the function create_trainingdata_bc_dict under bc_trainingdata.py
    # Previously this was a bug. We were loading in the actions in the wrong order when assigning from a list
    a = {'height': a[0], 'distance': a[1], 'heading': a[2], 'wrist_angle': a[3], 'wrist_rotation': a[4], 'gripper': a[5]}
    # Modified from_normalized vector to work with dict approach
    pos_target = RobotPosition.from_normalized_vector_dict(exp_robot_controller,a)
    print(f"Target position: {pos_target}", flush=True)
    if not robot_controller:
        return "0"
    # we assume that we have the robot
    # ask the user whether we indeed want to continue moving?
    pos_current = robot_controller.get_position()
    # Modified empirical_distance to work with dict approach
    distance = pos_current.empirical_distance_dict(exp_robot_controller, pos_target)
    query = f"Next position: {pos_target} at distance={distance} Proceed with the move? (A/Y/N/Q)"
    ans = input(query)

    # create an artificial target, changing the heading, just in case

    if ans.lower() == "y": # move towards the target, a safe amound
        pos_target_safe = RobotPosition(exp_robot_controller, pos_current.values)

        # Created a loop to update all of the actions rather than just the two previously
        # Removed diff calculation as it would produce outputs which different than the training data
        #   This causes the controller to become stagnant
        for key in pos_target_safe.values:
            # diff = (pos_target[key] - pos_target_safe[key])
            # diff = min(0.1, max(diff, -0.1))
            pos_target_safe[key] = pos_target[key]

        robot_controller.move(pos_target_safe)
        ""
        return "y"
    elif ans.lower() == "a": # make an artificial move, just to see that we are controlling the robot
        pos_target_artificial = RobotPosition(exp_robot_controller, pos_current.values)
        pos_target_artificial["heading"] = pos_target_artificial["heading"] + 3
        robot_controller.move(pos_target_artificial)
    elif ans.lower() == "q":
        return "q"


In [8]:
def run_robot(model, robot_controller, exp_robot_controller, camera_controller, controller_interval = 0.1):
    """Robot running loop. Loads the image from the camera, runs the model, and executes the recommended code."""
    # last_interval = 0
    controller_interval = 0.1

    transform = sp_helper.get_transform_to_sp(exp_sp)
    # the backlog of the last sequence length steps
    z_seq = []

    while True:
        start_time = time.time() 
        # Let us assume that the first image here is the one
        toexit = camera_controller.update()
        if toexit:
            break
        # FIXME: this should be made different... based on the run
        # image = camera_controller.images[exp["control_camera"]]
        active_camera = camera_controller.exp["active_camera_list"][0]
        image = camera_controller.images["dev" + str(active_camera)]
        # FIXME: this is totally won't work like this
        # there should be a notebook with the input processing???
        sensor_readings = sp_helper.load_capture_to_tensor(image, transform, device)
        z = sp.process(sensor_readings[0])
        # Commented this out as it pollutes the output
        # print(z)
        if exp_bc["sequence_length"]: # no sequence processsing
            z_seq.append(z)
            if len(z_seq) > exp_bc["sequence_length"]:
                z_seq.pop(0)
            if len(z_seq) < exp_bc["sequence_length"]:
                ## we don't have enough in the queue, wait.
                print("Not acting, waiting on the queue", flush=True)
                z_tensor = None
                pass
            else:
                # FIXME: this should be applied online, not step by step
                z_seq_np = np.array(z_seq)
                z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(device)
        else:
            z_tensor = torch.tensor(z).unsqueeze(0).to(device)
        if z_tensor == None:
            continue
        if not model.stochastic:                
            a_pred = model(z_tensor)
        else:
            a_pred = model.forward_and_sample(z_tensor)

        # a_pred is converted to a list as it's easier to work with in the safe_robot_control function
        retval = safe_robot_control(a_pred[0].tolist(), robot_controller, exp_robot_controller)
        if retval == "q":
            break

        # ensuring that the execution happens 
        end_time = time.time() 
        execution_time = end_time - start_time 
        # last_interval = execution_time
        time_to_sleep = max(0.0, controller_interval - execution_time) 
        time.sleep(time_to_sleep) 

In [9]:
run_robot(model = model, robot_controller = robot_controller, exp_robot_controller = exp_robot_controller, camera_controller = camera_controller)

Not acting, waiting on the queue
Not acting, waiting on the queue
Not acting, waiting on the queue
Not acting, waiting on the queue
Not acting, waiting on the queue


_IceTransSocketUNIXConnect: Cannot connect to non-local host boloni-System-Product-Name
_IceTransSocketUNIXConnect: Cannot connect to non-local host boloni-System-Product-Name
Qt: Session management error: Could not open network socket


Not acting, waiting on the queue
Not acting, waiting on the queue
Not acting, waiting on the queue
Not acting, waiting on the queue
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
Target position: Position: 
 height:5.63
 distance:5.04
 heading:0.14
 wrist_angle:-21.64
 wrist_rotation:77.69
 gripper:110.88

{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
Target position: Position: 
 height:5.53
 distance:5.16
 heading:0.14
 wrist_angle:-31.75
 wrist_rotation:78.08
 gripper:106.79

{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
Target position: Position: 
 height:5.56
 distance:5.22
 heading:0.14
 wrist_angle:-21.02
 wrist_rotation:77.22
 gripper:102.32

{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
Target position: Position: 
 height:5

In [ ]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()